## Libraries


In [13]:
# import libraries
import csv
import datetime
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import dates as mpl_dates
import seaborn as sns

In [3]:
warnings.filterwarnings(action = 'ignore')

## Data Preprocessing

In [6]:
df = pd.read_csv('data/Sunspots.csv')
del df['Unnamed: 0']


In [7]:
df.head()


,Date,Monthly Mean Total Sunspot Number
0,1749-01-31,96.7
1,1749-02-28,104.3
2,1749-03-31,116.7
3,1749-04-30,92.8
4,1749-05-31,141.7


In [8]:
timeseries = df
timeseries['Date'] = pd.to_datetime(df['Date'])
timeseries = df.set_index(df['Date'])
del timeseries['Date']
timeseries.head()


,Monthly Mean Total Sunspot Number
Date,
1749-01-31,96.7
1749-02-28,104.3
1749-03-31,116.7
1749-04-30,92.8
1749-05-31,141.7


In [10]:
def plot_series(time, series, col = 'dodgerblue', lab = 'original', format="-", start=0, end=None):
    plt.style.use('seaborn')
    plt.plot(time[start:end], series[start:end], format, color = col, label = lab)
    plt.xlabel("Time")
    plt.ylabel("Series")
    # display the grid
    plt.grid(True)
    # got current figure, then autoformat date
    plt.gcf().autofmt_xdate() 
    # format datetime
    date_formate = mpl_dates.DateFormatter('%b/%d/%Y') 
    # set the format to out x-axis, gca is the get current axis
    plt.gca().xaxis.set_major_formatter(date_formate)
    plt.tight_layout() 
    plt.legend(loc = 'best')